In [1]:
import pandas as pd

In [2]:
# Загружаю датасет по депозитам

deposit_data = pd.read_csv('data/deposit_data.csv', index_col=0)

In [3]:
deposit_data = deposit_data.reset_index().drop('Unnamed: 0', axis=1)

In [4]:
deposit_data = deposit_data.drop('index', axis=1)

In [5]:
deposit_data.head()

,user_phone,campaign_cost,campaign_type_click,campaign_type_conversion,campaign_type_show,source_medium_android / mobile_app,source_medium_bing / organic,source_medium_email / email,source_medium_finance.rambler.ru / cpm,source_medium_google / organic,...,event_type_conversion,event_type_show,product_type_deposits,product_type_loan,product_type_mortgage,event_status_approve,event_status_deal,event_status_payout,event_status_reject,event_status_sale
0,79123364690,0.0,1,0,1,0,0,0,0,0,...,0,1,1,0,0,0,0,0,0,0
1,79108520894,0.0,0,0,1,0,0,0,0,0,...,0,1,1,0,0,0,0,0,0,0
2,79104979450,0.0,0,0,1,0,0,0,0,0,...,0,1,1,0,0,0,0,0,0,0
3,79101386075,0.0,0,0,1,0,0,0,0,0,...,0,3,1,2,0,0,0,0,0,0
4,79106677780,0.0,0,0,1,0,0,0,0,0,...,0,1,2,0,0,0,0,0,0,0


In [6]:
deposit_data.sort_values(by='campaign_type_conversion', ascending=False).head(5)

,user_phone,campaign_cost,campaign_type_click,campaign_type_conversion,campaign_type_show,source_medium_android / mobile_app,source_medium_bing / organic,source_medium_email / email,source_medium_finance.rambler.ru / cpm,source_medium_google / organic,...,event_type_conversion,event_type_show,product_type_deposits,product_type_loan,product_type_mortgage,event_status_approve,event_status_deal,event_status_payout,event_status_reject,event_status_sale
611084,79135447910,9943.155,2,7,2,0,1,1,0,1,...,0,1,1,0,0,0,0,0,0,0
1140786,79174181594,11221.160,2,7,2,0,0,1,0,2,...,0,2,1,1,0,0,0,0,0,0
1000976,79165457436,1619.050,2,6,2,1,0,0,0,2,...,0,1,1,0,0,0,0,0,0,0
659633,79137117046,352.380,2,6,2,0,0,0,0,0,...,0,1,1,0,0,0,0,0,0,0
1098791,79174514473,3117.470,2,6,3,0,0,1,0,4,...,1,2,4,0,0,0,1,0,0,0


In [7]:
deposit_data.columns

Index(['user_phone', 'campaign_cost', 'campaign_type_click',
       'campaign_type_conversion', 'campaign_type_show',
       'source_medium_android / mobile_app', 'source_medium_bing / organic',
       'source_medium_email / email', 'source_medium_finance.rambler.ru / cpm',
       'source_medium_google / organic', 'source_medium_ios / mobile_app',
       'source_medium_not-set / none', 'source_medium_pikabu.ru / cpm',
       'source_medium_regular / email', 'source_medium_regular / sms',
       'source_medium_ru.android.bankproduct / mobile_app',
       'source_medium_ru.android.bankproduct_rustore / mobile_app',
       'source_medium_trigger / sms', 'source_medium_vk / cpc',
       'source_medium_web.telegram.org / referral ',
       'source_medium_ya.ru / cpc', 'source_medium_yahoo / organic',
       'source_medium_yandex / cpc', 'source_medium_yandex / organic',
       'source_medium_zen.yandex.ru / cpm', 'interface_api', 'interface_app',
       'interface_web', 'payout', 'event_typ

In [8]:
# Суммы касаний кликов, показов и конверсий

sum_of_touches = deposit_data[[
    'campaign_type_click', 'campaign_type_show', 'campaign_type_conversion']].sum()

In [9]:
sum_of_touches

campaign_type_click          492954
campaign_type_show          1971691
campaign_type_conversion     108408
dtype: int64

In [10]:
# Сумма показов без конверсий

sum_of_touches_wo_conversions = sum_of_touches['campaign_type_click'] + \
    sum_of_touches['campaign_type_show']

In [11]:
sum_of_touches_wo_conversions

2464645

In [12]:
deposit_data_by_campaign_type_conversions = deposit_data.loc[
    deposit_data['campaign_type_conversion'] > 0][[
        'campaign_type_click', 'campaign_type_show', 'campaign_type_conversion']]

In [13]:
deposit_data_click_show_conversion = deposit_data_by_campaign_type_conversions.sum()

In [14]:
# Сумма касаний, которые привели к конверсиям

sum_of_touches_lead_to_conversions = deposit_data_by_campaign_type_conversions.sum(
)['campaign_type_click'] + deposit_data_by_campaign_type_conversions.sum()['campaign_type_show']

In [15]:
sum_of_touches_lead_to_conversions

103583

In [16]:
# Процент касаний, который привел к конверсиям:

conversion_touches_percent = round(
    sum_of_touches_lead_to_conversions / sum_of_touches_wo_conversions * 100, 3)

In [17]:
conversion_touches_percent

4.203

In [18]:
# Общие затраты на рекламу для депозитов

total_expenditures = round(deposit_data['campaign_cost'].sum(), 2)

In [19]:
total_expenditures

61175081.23

In [20]:
# Общий доход как сумма столбца payout

total_payout = deposit_data['payout'].sum()

In [21]:
total_payout

67871852.94539998

In [22]:
# Общая прибыль от депозитов:

gross_margin = round(total_payout - total_expenditures, 3)

In [23]:
gross_margin

6696771.715

In [24]:
# Процент прибыли

gross_margin_percent = round(gross_margin / total_expenditures * 100, 3)

In [25]:
gross_margin_percent

10.947

In [26]:
# Количество одобрений банком

bank_positive_decisions = deposit_data.loc[(deposit_data['event_status_approve'] > 0) | 
    (deposit_data['event_status_deal'] > 0) |
    (deposit_data['event_status_payout'] > 0) |
    (deposit_data['event_status_sale'] > 0)
    ][['event_status_approve', 'event_status_deal',
                                              'event_status_payout', 'event_status_reject', 'event_status_sale']].sum()

In [27]:
bank_positive_decisions

event_status_approve    28395
event_status_deal       50566
event_status_payout       190
event_status_reject     12142
event_status_sale       17071
dtype: int64

In [28]:
# Общая сумма положительных решений банков

bank_positive_decisions_sum = bank_positive_decisions[['event_status_approve', 'event_status_deal',
                         'event_status_payout', 'event_status_sale']].sum() - bank_positive_decisions['event_status_reject']

In [29]:
# Процент одобрений банком от общего числа конверсий

bank_positive_decisions_percent = round(
    bank_positive_decisions_sum / deposit_data_click_show_conversion['campaign_type_conversion'].sum() * 100, 3)

In [30]:
bank_positive_decisions_percent

77.559

Процентное соотношение различных элементов

In [31]:
api_sums = deposit_data[['interface_api', 'interface_app', 'interface_web']].sum()

In [32]:
api_sums

interface_api     547109
interface_app    1294879
interface_web     731065
dtype: int64

In [33]:
for col in ['interface_api', 'interface_app', 'interface_web']:
    print(col, ': ', round(api_sums[col] / api_sums.sum() * 100, 3))

interface_api :  21.263
interface_app :  50.325
interface_web :  28.412


In [34]:
medium_sources = [
    'source_medium_android / mobile_app', 'source_medium_bing / organic',
    'source_medium_email / email', 'source_medium_finance.rambler.ru / cpm',
    'source_medium_google / organic', 'source_medium_ios / mobile_app',
    'source_medium_not-set / none', 'source_medium_pikabu.ru / cpm',
    'source_medium_regular / email', 'source_medium_regular / sms',
    'source_medium_ru.android.bankproduct / mobile_app',
    'source_medium_ru.android.bankproduct_rustore / mobile_app',
    'source_medium_trigger / sms', 'source_medium_vk / cpc',
    'source_medium_web.telegram.org / referral ',
    'source_medium_ya.ru / cpc', 'source_medium_yahoo / organic',
    'source_medium_yandex / cpc', 'source_medium_yandex / organic',
    'source_medium_zen.yandex.ru / cpm'
]

In [35]:
medium_sums = deposit_data[medium_sources].sum()

In [36]:
# Сумма показов рекламы для каждой рекламной площадки

medium_sums

source_medium_android / mobile_app                           128856
source_medium_bing / organic                                  26258
source_medium_email / email                                  128551
source_medium_finance.rambler.ru / cpm                        25667
source_medium_google / organic                               567643
source_medium_ios / mobile_app                               333726
source_medium_not-set / none                                  25680
source_medium_pikabu.ru / cpm                                 25957
source_medium_regular / email                                 25896
source_medium_regular / sms                                   25785
source_medium_ru.android.bankproduct / mobile_app             76682
source_medium_ru.android.bankproduct_rustore / mobile_app     25781
source_medium_trigger / sms                                   25729
source_medium_vk / cpc                                       128829
source_medium_web.telegram.org / referral       

In [37]:
# Процент показов в каждой рекламной площадке

for col in medium_sources:

    print(col, ': ', round(medium_sums[col] / medium_sums.sum() * 100, 3))

source_medium_android / mobile_app :  5.008
source_medium_bing / organic :  1.02
source_medium_email / email :  4.996
source_medium_finance.rambler.ru / cpm :  0.998
source_medium_google / organic :  22.061
source_medium_ios / mobile_app :  12.97
source_medium_not-set / none :  0.998
source_medium_pikabu.ru / cpm :  1.009
source_medium_regular / email :  1.006
source_medium_regular / sms :  1.002
source_medium_ru.android.bankproduct / mobile_app :  2.98
source_medium_ru.android.bankproduct_rustore / mobile_app :  1.002
source_medium_trigger / sms :  1.0
source_medium_vk / cpc :  5.007
source_medium_web.telegram.org / referral  :  1.002
source_medium_ya.ru / cpc :  3.012
source_medium_yahoo / organic :  1.004
source_medium_yandex / cpc :  9.981
source_medium_yandex / organic :  22.951
source_medium_zen.yandex.ru / cpm :  0.993
